In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import haversine_distances

In [2]:
df = pd.read_excel("Canadian Radio Playlist_V.21.xlsx", "Campus Radio Charts")
df.head(1)

/home/owen/Programs/anaconda3/envs/lighthouse/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M-MUSIC,A-ARTIST,P-PERFORMANCE,L-LYRICS,Unnamed: 24
0,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.5,4,Breakestra,US,"Los Angeles, CA",...,Indie,English,Yes,Mixed Group,Male Group,No,No,No,No,NaN


In [3]:
# Drop empty column
clean_df = df.drop('Unnamed: 24', axis=1)
# Convert place holder strings into nan values
clean_df.replace(['',' ', '-', '?','- ','? '], np.nan, inplace=True)

In [4]:
clean_df.nunique()

WEEK OF                          267
STATION                           26
STATION CITY                      21
STATION PROVINCE                   7
STATION LATITUDE                  21
STATION LONGITUDE                 21
CHART POSITION                    10
ARTIST NAME(S)                  2165
ARTIST COUNTRY                     6
ARTIST HOME CITY                 355
ARTIST HOME LATITUDE             302
ARTIST HOME  LONGITUDE           316
KM DISTANCE (HOME - STATION)    1777
ALBUM NAME                      2711
LABEL NAME                      1125
LABEL TYPE                         5
LANGUAGE OF MUSIC                 19
VISIBLE ETHNIC MINORITY            3
CENSUS RACE CLASSIFICATION        20
ARTIST GENDER                      9
M-MUSIC                            2
A-ARTIST                           2
P-PERFORMANCE                      2
L-LYRICS                           2
dtype: int64

In [5]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

STATION CITY                       1
STATION PROVINCE                   1
STATION LATITUDE                   1
STATION LONGITUDE                 11
CHART POSITION                    25
ARTIST NAME(S)                     1
ARTIST COUNTRY                   395
ARTIST HOME CITY                1746
ARTIST HOME LATITUDE            1872
ARTIST HOME  LONGITUDE          1905
KM DISTANCE (HOME - STATION)    1872
ALBUM NAME                         1
LABEL NAME                         1
LABEL TYPE                        33
LANGUAGE OF MUSIC                190
VISIBLE ETHNIC MINORITY          341
CENSUS RACE CLASSIFICATION       340
ARTIST GENDER                    288
M-MUSIC                          188
A-ARTIST                         188
P-PERFORMANCE                    188
L-LYRICS                         191
dtype: int64

# Clean 'WEEK OF' to datetime

In [6]:
clean_df['WEEK OF']

0        2006-01-10 00:00:00
1        2006-01-10 00:00:00
2        2006-01-10 00:00:00
3        2006-01-10 00:00:00
4        2006-01-10 00:00:00
                ...         
35854               3/2/1010
35855               3/2/1010
35856               3/2/1010
35857               3/2/1010
35858               3/2/1010
Name: WEEK OF, Length: 35859, dtype: object

In [7]:
dates = clean_df['WEEK OF'].astype(str).str.split().str[0]
dates

0        2006-01-10
1        2006-01-10
2        2006-01-10
3        2006-01-10
4        2006-01-10
            ...    
35854      3/2/1010
35855      3/2/1010
35856      3/2/1010
35857      3/2/1010
35858      3/2/1010
Name: WEEK OF, Length: 35859, dtype: object

In [8]:
#Looking in the dataset it appears 2010-03-02 is a missing week, which resembles this typo
dates = dates.replace('3/2/1010', '2010-03-02')
clean_df['WEEK OF'] = pd.to_datetime(dates)

# Clean Station Information

In [9]:
def replace_nans(key:str, columns:list):
    """Given a key column and a list of columns to fix,
    this function will fill in nan values with the assumption
    that all values in the columns to fix will be the same
    for any given value in the key column.
    """
    
    # Create list of all given key values that have any samples with nan values in given columns
    for key_value in df[(df[columns].transpose().isna().any())][key].unique():
        #Create filters
        any_nan = (clean_df[columns].transpose().isna().any()) & (clean_df[key] == key_value)
        all_nan = (df[columns].transpose().isna().all()) & (df[key] == key_value)
        #Get values from a row that contains not nan values
        values = df[~all_nan].head(1)[columns].iloc[0]
        #Replace nans with values
        clean_df.loc[any_nan, columns] = values.tolist()

    

In [10]:
replace_nans('STATION', ['STATION CITY',
                         'STATION PROVINCE',
                         'STATION LATITUDE',
                         'STATION LONGITUDE']  )

In [11]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                    25
ARTIST NAME(S)                     1
ARTIST COUNTRY                   395
ARTIST HOME CITY                1746
ARTIST HOME LATITUDE            1872
ARTIST HOME  LONGITUDE          1905
KM DISTANCE (HOME - STATION)    1872
ALBUM NAME                         1
LABEL NAME                         1
LABEL TYPE                        33
LANGUAGE OF MUSIC                190
VISIBLE ETHNIC MINORITY          341
CENSUS RACE CLASSIFICATION       340
ARTIST GENDER                    288
M-MUSIC                          188
A-ARTIST                         188
P-PERFORMANCE                    188
L-LYRICS                         191
dtype: int64

# Clean Artist Information

In [12]:
df[df['ARTIST NAME(S)'].isna()].values

array([[datetime.datetime(2007, 7, 17, 0, 0), 'CFUV  ', 'Victoria', 'BC',
        48.43, -123.35, 10, nan, '-', nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

In [13]:
# Drop row with no artist data
clean_df = clean_df[~clean_df['ARTIST NAME(S)'].isna()]

In [14]:
replace_nans('ARTIST NAME(S)', ['ARTIST HOME CITY',
                                'ARTIST HOME LATITUDE',
                                'ARTIST HOME  LONGITUDE',
                                'VISIBLE ETHNIC MINORITY', 
                                'CENSUS RACE CLASSIFICATION', 
                                'ARTIST GENDER',
                                'M-MUSIC',
                                'A-ARTIST ',
                                'P-PERFORMANCE',
                                'L-LYRICS'
                                ]  )

In [15]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                    25
ARTIST COUNTRY                   394
ARTIST HOME LATITUDE              53
ARTIST HOME  LONGITUDE            53
KM DISTANCE (HOME - STATION)    1871
LABEL TYPE                        32
LANGUAGE OF MUSIC                189
VISIBLE ETHNIC MINORITY           88
CENSUS RACE CLASSIFICATION        88
ARTIST GENDER                     37
M-MUSIC                            6
A-ARTIST                           6
P-PERFORMANCE                      6
L-LYRICS                           6
dtype: int64

### Handling remaining nans:

In [16]:
clean_df['ARTIST COUNTRY'].unique()

array(['US', 'UK', 'Int', 'Cdn', 'US/CDN', nan, 'int'], dtype=object)

In [17]:
clean_df['ARTIST GENDER'].unique()

array(['Male Group', 'Male', 'Female', 'Mixed Group', 'Male ',
       'Female Group', 'male', nan, 'Mixed Group '], dtype=object)

In [18]:
clean_df['VISIBLE ETHNIC MINORITY'].unique()

array(['Yes', 'No', nan], dtype=object)

In [19]:
clean_df['CENSUS RACE CLASSIFICATION'].unique()

array(['Mixed Group', 'White', 'Black', 'Hispanic', 'Asian Other',
       'Asian', 'Native Canadian', nan, 'Middle Eastern',
       'Native American', 'Unidentified', 'Asian Indian', 'Jewish',
       'Romany', 'Other Asian', 'Indian', 'Indian Asian', 'Inuit',
       'Metis', 'East Asian'], dtype=object)

In [20]:
clean_df['M-MUSIC'].unique()

array(['No', 'Yes', nan], dtype=object)

In [21]:
# Columns with NO existing category suitable for nan replacement
for key in ['ARTIST COUNTRY', 'ARTIST GENDER', 'VISIBLE ETHNIC MINORITY', 'CENSUS RACE CLASSIFICATION']:
    clean_df[key].replace(np.nan, 'unknown', inplace=True)
# Columns with existing category suitable for nan replacement
for key in ['M-MUSIC', 'A-ARTIST ', 'P-PERFORMANCE', 'L-LYRICS']:
    clean_df[key].replace(np.nan, 'No', inplace=True)





### Lattitude / Longitude:

In [22]:
clean_df[clean_df['ARTIST HOME LATITUDE'].isna()]['ARTIST NAME(S)'].unique()

array(['Geckoturner ', 'Nas & Damian Marley ', 'Non + Herrmutt Lobby '],
      dtype=object)

In [23]:
clean_df[clean_df['ARTIST HOME LATITUDE'].isna()]['ARTIST HOME CITY'].unique()

array(['Spain', 'New York, NY/Kingston, JAMAICA',
       'Los Angeles, CA/Belgium'], dtype=object)

In [24]:
home_location_columns = ['ARTIST HOME LATITUDE', 'ARTIST HOME  LONGITUDE']

In [25]:
# Replace 'Spain' with coordinates of Madrid
values = [40.416775 , -3.703790]
clean_df.loc[clean_df['ARTIST NAME(S)'] == 'Geckoturner ', home_location_columns] = values

# Replace 'New York, NY/Kingston, JAMAICA' with New York
values = clean_df.loc[clean_df['ARTIST HOME CITY'].str.contains('New York'), home_location_columns].head(1).iloc[0].to_list()
clean_df.loc[clean_df['ARTIST NAME(S)'] == 'Nas & Damian Marley ', home_location_columns] = values

# Replace 'Los Angeles, CA/Belgium' with Los Angeles
values = clean_df.loc[clean_df['ARTIST HOME CITY'].str.contains('Los Angeles'), home_location_columns].head(1).iloc[0].to_list()
clean_df.loc[clean_df['ARTIST NAME(S)'] == 'Non + Herrmutt Lobby ', home_location_columns] = values


In [26]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                    25
KM DISTANCE (HOME - STATION)    1871
LABEL TYPE                        32
LANGUAGE OF MUSIC                189
dtype: int64

# Clean album infromation

In [27]:
replace_nans('ALBUM NAME', ['LANGUAGE OF MUSIC'])

In [28]:
replace_nans('LABEL NAME', ['LABEL TYPE'])

In [29]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                    25
KM DISTANCE (HOME - STATION)    1871
LABEL TYPE                        21
LANGUAGE OF MUSIC                177
dtype: int64

In [30]:
# For language matches that could not be made with albums, match with artists instead
replace_nans('ARTIST NAME(S)',['LANGUAGE OF MUSIC'])

In [31]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                    25
KM DISTANCE (HOME - STATION)    1871
LABEL TYPE                        21
LANGUAGE OF MUSIC                 81
dtype: int64

In [32]:
clean_df['LABEL TYPE'].unique()

array(['Indie', 'Major', 'Self', nan, 'indie', 'English'], dtype=object)

In [33]:
clean_df['LABEL TYPE'].replace(np.nan, 'unknown', inplace=True)
#Consulting the dataframe, 'English' is a singular typo
clean_df['LABEL TYPE'].replace('English', 'indie', inplace=True)

In [34]:
clean_df['LANGUAGE OF MUSIC'].unique()

array(['English', 'French', nan, 'Multi', 'Spanish', 'Portuguese',
       'English/Patois', 'Basque', 'Other', 'english', 'English/Arabic',
       'German', 'other', 'e', 'English/Zulu', 'English/French', 'Yes',
       'Various', 'Punjabi', 'Creole'], dtype=object)

In [35]:
clean_df['LANGUAGE OF MUSIC'].replace(np.nan, 'unknown', inplace=True)
clean_df['LANGUAGE OF MUSIC'].replace('e', 'english', inplace=True)

In [36]:
clean_df['CHART POSITION'].unique()

array([ 4.,  5.,  7.,  8.,  9., 10.,  3.,  2.,  6.,  1., nan])

In [37]:
nan_filter = clean_df['CHART POSITION'].isna()
artists_with_nan_chart_numbers = clean_df[ nan_filter]['ARTIST NAME(S)']
artists_with_nan_chart_numbers.head(4)

552      Psyche Origami 
1179          Aceyalone 
2063    Dilated Peoples 
3048          Jay Bizzy 
Name: ARTIST NAME(S), dtype: object

In [38]:
clean_df.loc[clean_df['ARTIST NAME(S)'] == 'Aceyalone ', 'CHART POSITION'].head(10)

882      4.0
1031     2.0
1039     4.0
1067    10.0
1161     2.0
1169     3.0
1179     NaN
1199    10.0
1232     7.0
1242     6.0
Name: CHART POSITION, dtype: float64

In [39]:
# replace nan values in chart numbers with average chart position held by artist
for name in artists_with_nan_chart_numbers:
    values = clean_df.loc[clean_df['ARTIST NAME(S)'] == name, 'CHART POSITION']
    values.replace(np.nan, int(values.mean()), inplace=True)
    clean_df.loc[clean_df['ARTIST NAME(S)'] == name, 'CHART POSITION'] = values

In [40]:
clean_df['CHART POSITION'] = clean_df['CHART POSITION'].astype(float)

# Clean station -> artist distance

In [41]:
def haversine(row:pd.Series) -> float:
    X = [math.radians(row[0]), math.radians(row[1])]
    Y = [math.radians(row[2]), math.radians(row[3])]
    return (haversine_distances([X,Y]) * 6371)[0][1]


In [42]:
clean_df['KM DISTANCE (HOME - STATION)'] = clean_df[['STATION LATITUDE', 
                                                     'STATION LONGITUDE', 
                                                     'ARTIST HOME LATITUDE', 
                                                     'ARTIST HOME  LONGITUDE']
                                                    ].apply(haversine, axis=1)



In [43]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

Series([], dtype: int64)

# Clean strings

In [44]:
# Clean extra spaces from column names
clean_df.columns = clean_df.columns.str.strip()
clean_df.columns = clean_df.columns.str.replace('\s+', ' ', regex=True)

In [45]:
# Clean string values
string_columns = clean_df.select_dtypes(exclude=[np.number, np.datetime64]).columns

for column in string_columns:
    cleaned = clean_df[column].astype(str)
    cleaned = cleaned.str.strip()
    cleaned = cleaned.str.replace('\s+', ' ', regex=True)
    cleaned = cleaned.str.lower()
    
    clean_df[column] = cleaned
    

## Add canadian-content status (met by having at least 2 MAPL ratings)

In [48]:
clean_df.rename(columns={"M-MUSIC":"M", "A-ARTIST": "A", "P-PERFORMANCE":"P","L-LYRICS":"L"}, inplace = True)


In [52]:
def is_can_con(row:pd.Series) -> str:
    """Given row containing M A L P, determines
    Canadian Content status
    """
    if row.str.count("yes").sum() >=2:
        return 'yes'
    else:
        return 'no'

In [56]:
clean_df['CANADIAN CONTENT'] = clean_df[['M','A','L','P']].apply(is_can_con, axis=1)

In [58]:
clean_df.to_csv('clean_data.csv')

In [61]:
clean_df.head(10)

,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M,A,P,L,CANADIAN CONTENT
0,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,4.0,breakestra,us,"los angeles, ca",...,indie,english,yes,mixed group,male group,no,no,no,no,no
1,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,5.0,candy's .22,us,"los angeles, ca",...,indie,english,no,white,male group,no,no,no,no,no
2,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,7.0,dangerdoom,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no
3,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,8.0,blockhead,us,"new york, ny",...,indie,english,no,white,male,no,no,no,no,no
4,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,9.0,blackalicious,us,"sacramento, ca",...,indie,english,yes,black,male group,no,no,no,no,no
5,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,10.0,onry ozzborn,us,"seattle, wa",...,indie,english,yes,hispanic,male,no,no,no,no,no
6,2006-01-10,cjsw,calgary,ab,51.08,-114.08,3.0,various,us,"los angeles, ca",...,indie,english,yes,mixed group,male group,no,no,no,no,no
7,2006-01-10,cjsw,calgary,ab,51.08,-114.08,5.0,psyche origami,us,"atlanta, ga",...,indie,english,yes,mixed group,male group,no,no,no,no,no
8,2006-01-10,cjsw,calgary,ab,51.08,-114.08,7.0,various,us,"los angeles, ca",...,indie,english,yes,mixed group,male group,no,no,no,no,no
9,2006-01-10,cjsw,calgary,ab,51.08,-114.08,8.0,mike ladd,us,"boston, mass",...,indie,english,yes,black,male group,no,no,no,no,no
